# The Power of labeled data structures

***Purpose: Your data has labels; you should use them***

![](images/dataset-diagram.png)

Scientific data is inherently labeled. For example, time series data includes timestamps that label individual periods or points in time, spatial data has coordinates (e.g. longitude, latitude, elevation), and model or laboratory experiments are often identified by unique identifiers. The figure above provides an example of a labeled dataset. In this case the data is a map of global air temperature from a numeric weather model. The labels on this particular dataset are time (e.g. “2016-05-01”), longitude (x-axis), and latitude (y-axis).

----

### Outline
- Numpy index (in brief)
- Named dimensions/axes
- Coordinate labels
- Label based indexing
- Alignment

### Tutorial Duriation
10 minutes

### Going Further

Xarray Documentation on Indexing: http://xarray.pydata.org/en/latest/indexing.html

In [1]:
import xarray as xr


/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
# load a sample dataset
ds = xr.tutorial.load_dataset('air_temperature')
ds

<xarray.Dataset>
Dimensions:  (lat: 25, lon: 53, time: 2920)
Coordinates:
  * lat      (lat) float32 75.0 72.5 70.0 67.5 65.0 62.5 60.0 57.5 55.0 52.5 ...
  * lon      (lon) float32 200.0 202.5 205.0 207.5 210.0 212.5 215.0 217.5 ...
  * time     (time) datetime64[ns] 2013-01-01 2013-01-01T06:00:00 ...
Data variables:
    air      (time, lat, lon) float32 241.2 242.5 243.5 244.0 244.09999 ...
Attributes:
    Conventions:  COARDS
    title:        4x daily NMC reanalysis (1948)
    description:  Data is from NMC initialized reanalysis\n(4x/day).  These a...
    platform:     Model
    references:   http://www.esrl.noaa.gov/psd/data/gridded/data.ncep.reanaly...

## The old way (numpy positional indexing)

When working with numpy, indexing is done by position (slices/ranges/scalars). 

In [3]:
t = ds['air'].data  # numpy array
t

array([[[241.2    , 242.5    , 243.5    , ..., 232.79999, 235.5    ,
         238.59999],
        [243.79999, 244.5    , 244.7    , ..., 232.79999, 235.29999,
         239.29999],
        [250.     , 249.79999, 248.89   , ..., 233.2    , 236.39   ,
         241.7    ],
        ...,
        [296.6    , 296.19998, 296.4    , ..., 295.4    , 295.1    ,
         294.69998],
        [295.9    , 296.19998, 296.79   , ..., 295.9    , 295.9    ,
         295.19998],
        [296.29   , 296.79   , 297.1    , ..., 296.9    , 296.79   ,
         296.6    ]],

       [[242.09999, 242.7    , 243.09999, ..., 232.     , 233.59999,
         235.79999],
        [243.59999, 244.09999, 244.2    , ..., 231.     , 232.5    ,
         235.7    ],
        [253.2    , 252.89   , 252.09999, ..., 230.79999, 233.39   ,
         238.5    ],
        ...,
        [296.4    , 295.9    , 296.19998, ..., 295.4    , 295.1    ,
         294.79   ],
        [296.19998, 296.69998, 296.79   , ..., 295.6    , 295.5    ,
   

In [4]:
t.shape

(2920, 25, 53)

In [5]:
# extract a time-series for one spatial location
t[:, 10, 20]

array([269.79   , 267.69998, 269.9    , ..., 266.19   , 270.88998,
       268.38998], dtype=float32)

but wait, what labels go with `10` and `20`? Was that lat/lon or lon/lat? Where are the timestamps that go along with this time-series?

# Indexing with Xarray

xarray offers extremely flexible indexing routines that combine the best features of NumPy and pandas for data selection.

In [6]:
da = ds['air']

In [7]:
# numpy style indexing still works (but preserves the labels/metadata)
da[:, 10, 20]

<xarray.DataArray 'air' (time: 2920)>
array([269.79   , 267.69998, 269.9    , ..., 266.19   , 270.88998, 268.38998],
      dtype=float32)
Coordinates:
    lat      float32 50.0
    lon      float32 250.0
  * time     (time) datetime64[ns] 2013-01-01 2013-01-01T06:00:00 ...
Attributes:
    long_name:     4xDaily Air temperature at sigma level 995
    units:         degK
    precision:     2
    GRIB_id:       11
    GRIB_name:     TMP
    var_desc:      Air temperature
    dataset:       NMC Reanalysis
    level_desc:    Surface
    statistic:     Individual Obs
    parent_stat:   Other
    actual_range:  [185.16 322.1 ]

In [8]:
# Positional indexing using dimension names
da.isel(lat=10, lon=20)

<xarray.DataArray 'air' (time: 2920)>
array([269.79   , 267.69998, 269.9    , ..., 266.19   , 270.88998, 268.38998],
      dtype=float32)
Coordinates:
    lat      float32 50.0
    lon      float32 250.0
  * time     (time) datetime64[ns] 2013-01-01 2013-01-01T06:00:00 ...
Attributes:
    long_name:     4xDaily Air temperature at sigma level 995
    units:         degK
    precision:     2
    GRIB_id:       11
    GRIB_name:     TMP
    var_desc:      Air temperature
    dataset:       NMC Reanalysis
    level_desc:    Surface
    statistic:     Individual Obs
    parent_stat:   Other
    actual_range:  [185.16 322.1 ]

In [9]:
# Label-based indexing
da.sel(lat=50., lon=250.)

<xarray.DataArray 'air' (time: 2920)>
array([269.79   , 267.69998, 269.9    , ..., 266.19   , 270.88998, 268.38998],
      dtype=float32)
Coordinates:
    lat      float32 50.0
    lon      float32 250.0
  * time     (time) datetime64[ns] 2013-01-01 2013-01-01T06:00:00 ...
Attributes:
    long_name:     4xDaily Air temperature at sigma level 995
    units:         degK
    precision:     2
    GRIB_id:       11
    GRIB_name:     TMP
    var_desc:      Air temperature
    dataset:       NMC Reanalysis
    level_desc:    Surface
    statistic:     Individual Obs
    parent_stat:   Other
    actual_range:  [185.16 322.1 ]

In [10]:
# Nearest neighbor lookups
da.sel(lat=52.25, lon=251.8998, method='nearest')

<xarray.DataArray 'air' (time: 2920)>
array([262.69998, 263.19998, 270.9    , ..., 264.19   , 265.19   , 266.99   ],
      dtype=float32)
Coordinates:
    lat      float32 52.5
    lon      float32 252.5
  * time     (time) datetime64[ns] 2013-01-01 2013-01-01T06:00:00 ...
Attributes:
    long_name:     4xDaily Air temperature at sigma level 995
    units:         degK
    precision:     2
    GRIB_id:       11
    GRIB_name:     TMP
    var_desc:      Air temperature
    dataset:       NMC Reanalysis
    level_desc:    Surface
    statistic:     Individual Obs
    parent_stat:   Other
    actual_range:  [185.16 322.1 ]

In [11]:
# all of these indexing methods work on the dataset too, e.g.:
ds.sel(lat=52.25, lon=251.8998, method='nearest')

<xarray.Dataset>
Dimensions:  (time: 2920)
Coordinates:
    lat      float32 52.5
    lon      float32 252.5
  * time     (time) datetime64[ns] 2013-01-01 2013-01-01T06:00:00 ...
Data variables:
    air      (time) float32 262.69998 263.19998 270.9 274.1 273.29 270.6 ...
Attributes:
    Conventions:  COARDS
    title:        4x daily NMC reanalysis (1948)
    description:  Data is from NMC initialized reanalysis\n(4x/day).  These a...
    platform:     Model
    references:   http://www.esrl.noaa.gov/psd/data/gridded/data.ncep.reanaly...

## Vectorized indexing

Like numpy and pandas, xarray supports indexing many array elements at once in a vectorized manner.

In [12]:
# generate a coordinates for a transect of points
lat_points = xr.DataArray([52, 52.5, 53], dims='points')
lon_points = xr.DataArray([250, 250, 250], dims='points')

In [13]:
# nearest neighbor selection along the transect
da.sel(lat=lat_points, lon=lon_points, method='nearest')

<xarray.DataArray 'air' (time: 2920, points: 3)>
array([[269.5    , 269.5    , 269.5    ],
       [269.29   , 269.29   , 269.29   ],
       [273.69998, 273.69998, 273.69998],
       ...,
       [267.49   , 267.49   , 267.49   ],
       [269.29   , 269.29   , 269.29   ],
       [268.69   , 268.69   , 268.69   ]], dtype=float32)
Coordinates:
    lat      (points) float32 52.5 52.5 52.5
    lon      (points) float32 250.0 250.0 250.0
  * time     (time) datetime64[ns] 2013-01-01 2013-01-01T06:00:00 ...
Dimensions without coordinates: points
Attributes:
    long_name:     4xDaily Air temperature at sigma level 995
    units:         degK
    precision:     2
    GRIB_id:       11
    GRIB_name:     TMP
    var_desc:      Air temperature
    dataset:       NMC Reanalysis
    level_desc:    Surface
    statistic:     Individual Obs
    parent_stat:   Other
    actual_range:  [185.16 322.1 ]

## Alignment

xarray enforces alignment between index Coordinates (that is, coordinates with the same name as a dimension, marked by *) on objects used in binary operations.

In [14]:
da

<xarray.DataArray 'air' (time: 2920, lat: 25, lon: 53)>
array([[[241.2    , 242.5    , ..., 235.5    , 238.59999],
        [243.79999, 244.5    , ..., 235.29999, 239.29999],
        ...,
        [295.9    , 296.19998, ..., 295.9    , 295.19998],
        [296.29   , 296.79   , ..., 296.79   , 296.6    ]],

       [[242.09999, 242.7    , ..., 233.59999, 235.79999],
        [243.59999, 244.09999, ..., 232.5    , 235.7    ],
        ...,
        [296.19998, 296.69998, ..., 295.5    , 295.1    ],
        [296.29   , 297.19998, ..., 296.4    , 296.6    ]],

       ...,

       [[245.79   , 244.79   , ..., 243.98999, 244.79   ],
        [249.89   , 249.29   , ..., 242.48999, 244.29   ],
        ...,
        [296.29   , 297.19   , ..., 295.09   , 294.38998],
        [297.79   , 298.38998, ..., 295.49   , 295.19   ]],

       [[245.09   , 244.29   , ..., 241.48999, 241.79   ],
        [249.89   , 249.29   , ..., 240.29   , 241.68999],
        ...,
        [296.09   , 296.88998, ..., 295.69   , 

In [15]:
arr = da.isel(time=0, lat=slice(5, 10), lon=slice(7, 11))
arr

<xarray.DataArray 'air' (lat: 5, lon: 4)>
array([[265.1    , 264.9    , 265.5    , 266.5    ],
       [272.9    , 272.     , 271.79   , 271.1    ],
       [279.4    , 278.9    , 278.4    , 276.6    ],
       [279.5    , 279.9    , 280.5    , 279.9    ],
       [280.1    , 280.19998, 280.79   , 281.4    ]], dtype=float32)
Coordinates:
  * lat      (lat) float32 62.5 60.0 57.5 55.0 52.5
  * lon      (lon) float32 217.5 220.0 222.5 225.0
    time     datetime64[ns] 2013-01-01
Attributes:
    long_name:     4xDaily Air temperature at sigma level 995
    units:         degK
    precision:     2
    GRIB_id:       11
    GRIB_name:     TMP
    var_desc:      Air temperature
    dataset:       NMC Reanalysis
    level_desc:    Surface
    statistic:     Individual Obs
    parent_stat:   Other
    actual_range:  [185.16 322.1 ]

In [16]:
part = arr[:-1]
part

<xarray.DataArray 'air' (lat: 4, lon: 4)>
array([[265.1 , 264.9 , 265.5 , 266.5 ],
       [272.9 , 272.  , 271.79, 271.1 ],
       [279.4 , 278.9 , 278.4 , 276.6 ],
       [279.5 , 279.9 , 280.5 , 279.9 ]], dtype=float32)
Coordinates:
  * lat      (lat) float32 62.5 60.0 57.5 55.0
  * lon      (lon) float32 217.5 220.0 222.5 225.0
    time     datetime64[ns] 2013-01-01
Attributes:
    long_name:     4xDaily Air temperature at sigma level 995
    units:         degK
    precision:     2
    GRIB_id:       11
    GRIB_name:     TMP
    var_desc:      Air temperature
    dataset:       NMC Reanalysis
    level_desc:    Surface
    statistic:     Individual Obs
    parent_stat:   Other
    actual_range:  [185.16 322.1 ]

In [20]:
# default behavior is an "inner join"
(arr + part) /2

<xarray.DataArray 'air' (lat: 4, lon: 4)>
array([[265.1 , 264.9 , 265.5 , 266.5 ],
       [272.9 , 272.  , 271.79, 271.1 ],
       [279.4 , 278.9 , 278.4 , 276.6 ],
       [279.5 , 279.9 , 280.5 , 279.9 ]], dtype=float32)
Coordinates:
  * lat      (lat) float64 62.5 60.0 57.5 55.0
  * lon      (lon) float32 217.5 220.0 222.5 225.0
    time     datetime64[ns] 2013-01-01

In [18]:
# we can also use an outer join
with xr.set_options(arithmetic_join="outer"):
    print((arr + part) / 2)
    
# notice that missing values (nan) were inserted

<xarray.DataArray 'air' (lat: 5, lon: 4)>
array([[   nan,    nan,    nan,    nan],
       [279.5 , 279.9 , 280.5 , 279.9 ],
       [279.4 , 278.9 , 278.4 , 276.6 ],
       [272.9 , 272.  , 271.79, 271.1 ],
       [265.1 , 264.9 , 265.5 , 266.5 ]], dtype=float32)
Coordinates:
  * lat      (lat) float64 52.5 55.0 57.5 60.0 62.5
  * lon      (lon) float32 217.5 220.0 222.5 225.0
    time     datetime64[ns] 2013-01-01


## Broadcasting

DataArray objects are automatically align themselves (“broadcasting” in the numpy parlance) by dimension name instead of axis order. With xarray, you do not need to transpose arrays or insert dimensions of length 1 to get array operations to work, as commonly done in numpy with np.reshape() or np.newaxis.

In [21]:
mults = xr.DataArray([1, 2], [('mult', ['one', 'two'])])
mults

<xarray.DataArray (mult: 2)>
array([1, 2])
Coordinates:
  * mult     (mult) <U3 'one' 'two'

In [22]:
arr * mults

<xarray.DataArray (lat: 5, lon: 4, mult: 2)>
array([[[265.100006, 530.200012],
        [264.899994, 529.799988],
        [265.5     , 531.      ],
        [266.5     , 533.      ]],

       [[272.899994, 545.799988],
        [272.      , 544.      ],
        [271.790009, 543.580017],
        [271.100006, 542.200012]],

       [[279.399994, 558.799988],
        [278.899994, 557.799988],
        [278.399994, 556.799988],
        [276.600006, 553.200012]],

       [[279.5     , 559.      ],
        [279.899994, 559.799988],
        [280.5     , 561.      ],
        [279.899994, 559.799988]],

       [[280.100006, 560.200012],
        [280.199982, 560.399963],
        [280.790009, 561.580017],
        [281.399994, 562.799988]]])
Coordinates:
  * lat      (lat) float32 62.5 60.0 57.5 55.0 52.5
  * lon      (lon) float32 217.5 220.0 222.5 225.0
    time     datetime64[ns] 2013-01-01
  * mult     (mult) <U3 'one' 'two'